When I'm not even in my harshest

You can still get roasted

'cause Marsh is not Mellow

-- No Love

In [1]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

In [2]:
current_dir = os.path.dirname(os.path.abspath("Eminem"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

config.read(config_path)

client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [3]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

In [4]:
from spotipy.oauth2 import SpotifyClientCredentials

# Get credentials from the config file
client_id= '31d85b3f430e45caba7b97174cb75137'
client_secret = '5c6b719bcd8743b0ad1f8cd217ad1e1c'
client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
def collect_spotify_data():
    start_time = timer()
    data = []  


    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist:Eminem', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                # Get audio features for track
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    # Handle case where audio features are not available
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                # Get featured artists
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5)  # Retry after a short delay
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break  # Break the loop if there's a persistent error

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print(f"Elapsed time:", timekeeper(elapsed_time))
    return df_raw

# Collecting data from Spotify API 
df_spotify = collect_spotify_data()


Elapsed time: 4 minutes, 9 seconds


In [6]:
df_spotify

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Eminem,Without Me,7lQ8MOhq6IN2w8EYcFNSUk,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,86,True,0.908,290320,...,-2.827,1,0.0738,0.002860,0.000000,0.2370,0.6620,112.238,4,[]
1,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,The Marshall Mathers LP,6t7956yu5zYf5A829XRiHC,2000-05-23,84,True,0.949,284200,...,-4.244,0,0.0572,0.030200,0.000000,0.0454,0.7600,104.504,4,[]
2,Eminem,Till I Collapse,4xkOaSrkexMciUUogZKVTS,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,82,True,0.548,297787,...,-3.237,1,0.1860,0.062200,0.000000,0.0816,0.1000,171.447,4,[Nate Dogg]
3,Eminem,Mockingbird,561jH07mF1jHuk7KlaeF0s,Encore (Deluxe Version),1kTlYbs28MXw7hwO0NLYif,2004-11-12,85,True,0.637,250760,...,-3.798,1,0.2660,0.209000,0.000000,0.1560,0.2540,84.039,4,[]
4,Eminem,Lose Yourself,5Z01UMMf7V1o0MzF86s6WJ,Curtain Call: The Hits (Deluxe Edition),5qENHeCSlwWpEzb25peRmQ,2005-12-06,76,True,0.692,326467,...,-4.565,1,0.2570,0.008680,0.000423,0.3590,0.0612,171.374,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Eminem,Still Don't Give,2NPIFzhYie0mIAjxr5kw6a,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,21,False,0.635,250360,...,-2.914,0,0.4120,0.155000,0.000000,0.3640,0.6650,89.829,4,[]
996,Eminem,Role Model,4NQcaKgDJzxcUT4Ap44d9s,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,24,True,0.853,210400,...,-4.311,1,0.2550,0.020400,0.000008,0.0983,0.8240,91.911,4,[]
997,Eminem,Little Engine,5X1A8lPx3YRTNCV83NG57o,Music To Be Murdered By,2LWLgGY15e9im0KUaASnFa,2020-01-17,20,False,0.763,177293,...,-4.192,1,0.2390,0.022200,0.000000,0.0404,0.7250,155.079,4,[]
998,Eminem,Just Don't Give A Fuck - Instrumental,4FqtatWjhv6mG3kQFlUtu8,Just Don't Give A F***,3iBqm5kbcbMxqao0azRhZ0,1998-01-01,15,False,0.653,249933,...,-6.620,0,0.0515,0.000267,0.001090,0.1610,0.5440,85.429,4,[]


In [7]:
shady = df_spotify[df_spotify['artist_name']=="Eminem"].copy()
shady

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Eminem,Without Me,7lQ8MOhq6IN2w8EYcFNSUk,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,86,True,0.908,290320,...,-2.827,1,0.0738,0.002860,0.000000,0.2370,0.6620,112.238,4,[]
1,Eminem,The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR,The Marshall Mathers LP,6t7956yu5zYf5A829XRiHC,2000-05-23,84,True,0.949,284200,...,-4.244,0,0.0572,0.030200,0.000000,0.0454,0.7600,104.504,4,[]
2,Eminem,Till I Collapse,4xkOaSrkexMciUUogZKVTS,The Eminem Show,2cWBwpqMsDJC1ZUwz813lo,2002-05-26,82,True,0.548,297787,...,-3.237,1,0.1860,0.062200,0.000000,0.0816,0.1000,171.447,4,[Nate Dogg]
3,Eminem,Mockingbird,561jH07mF1jHuk7KlaeF0s,Encore (Deluxe Version),1kTlYbs28MXw7hwO0NLYif,2004-11-12,85,True,0.637,250760,...,-3.798,1,0.2660,0.209000,0.000000,0.1560,0.2540,84.039,4,[]
4,Eminem,Lose Yourself,5Z01UMMf7V1o0MzF86s6WJ,Curtain Call: The Hits (Deluxe Edition),5qENHeCSlwWpEzb25peRmQ,2005-12-06,76,True,0.692,326467,...,-4.565,1,0.2570,0.008680,0.000423,0.3590,0.0612,171.374,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,Eminem,Say Goodbye Hollywood,2l3giwDLI9KYsGh8dHCdjh,The Eminem Show,6FSYxVo82U1lXyDAKYqpvi,2002-05-26,20,False,0.753,272733,...,-5.008,0,0.1830,0.057800,0.000000,0.0907,0.6870,94.230,4,[]
995,Eminem,Still Don't Give,2NPIFzhYie0mIAjxr5kw6a,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,21,False,0.635,250360,...,-2.914,0,0.4120,0.155000,0.000000,0.3640,0.6650,89.829,4,[]
996,Eminem,Role Model,4NQcaKgDJzxcUT4Ap44d9s,The Slim Shady LP,2VFlCEu1NPLIbWjIw1lmjJ,1999-02-23,24,True,0.853,210400,...,-4.311,1,0.2550,0.020400,0.000008,0.0983,0.8240,91.911,4,[]
997,Eminem,Little Engine,5X1A8lPx3YRTNCV83NG57o,Music To Be Murdered By,2LWLgGY15e9im0KUaASnFa,2020-01-17,20,False,0.763,177293,...,-4.192,1,0.2390,0.022200,0.000000,0.0404,0.7250,155.079,4,[]


In [8]:
shady[shady['album_name']=='8 Mile (Music From And Inspired By The Motion Picture)']

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
127,Eminem,8 Mile,5NmL2zxnlt9GiTB2TQbvFM,8 Mile (Music From And Inspired By The Motion ...,353HFOqGHySp027oyr3aGs,2002-10-29,60,True,0.740,359400,...,-4.195,1,0.285,0.12500,0.000083,0.132,0.4890,89.981,4,[]
172,Eminem,Rabbit Run,30KKYkJ0lonfKI2eaLACnZ,8 Mile (Music From And Inspired By The Motion ...,353HFOqGHySp027oyr3aGs,2002-10-29,53,True,0.563,190067,...,-3.639,0,0.319,0.00260,0.000000,0.160,0.3360,95.392,4,[]
179,Eminem,Lose Yourself,6GkTKjv1XbFVdI2D8vvDPu,8 Mile (Music From And Inspired By The Motion ...,353HFOqGHySp027oyr3aGs,2002-10-29,53,True,0.684,320293,...,-4.596,1,0.273,0.00925,0.000659,0.288,0.0600,171.383,4,[]
232,Eminem,"Lose Yourself - From ""8 Mile"" Soundtrack",3u0W3gJQNV5gegMmntzby8,8 Mile (Music From And Inspired By The Motion ...,481Cwgl0ST0PknmPC0PH8r,2002-10-29,47,False,0.700,321960,...,-4.697,1,0.265,0.00491,0.000011,0.350,0.0588,171.404,4,[]
601,Eminem,"8 Mile - From ""8 Mile"" Soundtrack",4b8YZVxagUgUfpETkzzpiu,8 Mile (Music From And Inspired By The Motion ...,481Cwgl0ST0PknmPC0PH8r,2002-10-29,40,False,0.766,359080,...,-4.158,1,0.246,0.13000,0.000174,0.123,0.4570,89.986,4,[]
794,Eminem,"Rabbit Run - From ""8 Mile"" Soundtrack",1Lt9I2HY40gg7HVjh0Kw4N,8 Mile (Music From And Inspired By The Motion ...,481Cwgl0ST0PknmPC0PH8r,2002-10-29,31,False,0.568,190467,...,-5.730,0,0.305,0.00245,0.000002,0.140,0.3300,95.420,4,[]


In [9]:
albums = [
    'The Slim Shady LP',
    'The Marshall Mathers LP',
    'The Eminem Show',
    'Encore',
    'Relapse', 
    'Recovery',
    'The Marshall Mathers LP2', 
    'Revival',
    'Kamikaze',
    'Music To Be Murdered By', 
    'The Death of Slim Shady (Coup De Grâce)'
]

deluxe_editions = [
    'The Death of Slim Shady (Coup De Grâce): Expanded Mourner’s Edition',
    'The Marshall Mathers LP2 (Deluxe)', 
    'The Eminem Show (Expanded Edition)',
    'The Slim Shady LP (Expanded Edition)',
    'Encore (Deluxe Version)',
    'Recovery (Deluxe Edition)',
    'Relapse [Deluxe]'
]
soundtrack_albums = [
    'Southpaw (Music From And Inspired By The Motion Picture)',
    '8 Mile (Music From And Inspired By The Motion Picture)'
]

collaborative_albums = [
    'SHADYXV',
    'Eminem Presents The Re-Up'
]

reissued_albums = [
    'Relapse: Refill',
    'Music To Be Murdered By - Side B (Deluxe Edition)'
]

compilation_albums = [
    'Curtain Call: The Hits',
    'Curtain Call: The Hits (Deluxe Edition)',
    'Curtain Call 2'
]

singles = shady['track_name'].str.split().str[:3].str.join(' ') == shady['album_name'].str.split().str[:3].str.join(' ')
album = shady['album_name'].isin(albums)
deluxe = shady['album_name'].isin(deluxe_editions)
soundtrack = shady['album_name'].isin(soundtrack_albums)
collabo = shady['album_name'].isin(collaborative_albums)
reissued = shady['album_name'].isin(reissued_albums)
compilations = shady['album_name'].isin(compilation_albums)

conditions = [singles, album, deluxe, soundtrack, collabo, reissued,compilations]
choices = ['Single', 'Studio Album', 'Deluxe Album', 'Soundtrack Album', 'Collaborative Album', 'Reissued Albums', 'Compilations']
shady['type'] = np.select(conditions, choices, default='')

In [10]:
df = shady.copy()

In [80]:
df[['artist_name', 'track_name', 'album_name', 'type']].head()

,artist_name,track_name,album_name,type
0,Eminem,Without Me,The Eminem Show,Studio Album
1,Eminem,The Real Slim Shady,The Marshall Mathers LP,Studio Album
2,Eminem,Till I Collapse,The Eminem Show,Studio Album
3,Eminem,Mockingbird,Encore (Deluxe Version),Deluxe Album
5,Eminem,Lose Yourself,Curtain Call: The Hits (Deluxe Edition),Compilations


In [92]:
df.loc[(df['type'] == '') & (df['popularity'] > 60), 'type'] = 'Single'

In [97]:
notype = df[df['type']=='']
flush = notype.index
df= df.drop(flush)

In [98]:
df['mode'] = df['mode'].replace({0:'Minor', 1:'Major'})
df[['track_name', 'album_name', 'mode']].head()

,track_name,album_name,mode
0,Without Me,The Eminem Show,Major
1,The Real Slim Shady,The Marshall Mathers LP,Minor
2,Till I Collapse,The Eminem Show,Major
3,Mockingbird,Encore (Deluxe Version),Major
5,Lose Yourself,Curtain Call: The Hits (Deluxe Edition),Major


In [99]:
df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')

df['duration'] = df['duration_ms'].dt.components['minutes'].astype(str).str.zfill(2) + ':' + df['duration_ms'].dt.components['seconds'].astype(str).str.zfill(2)
df.drop(columns=['duration_ms'], inplace= True)
df[['track_name','duration']].head()

,track_name,duration
0,Without Me,04:50
1,The Real Slim Shady,04:44
2,Till I Collapse,04:57
3,Mockingbird,04:10
5,Lose Yourself,05:26


In [100]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['year'] = df['release_date'].dt.year
df['month'] = df['release_date'].dt.month_name()
df['day_of_the_week'] = df['release_date'].dt.day_name()

df[['track_name', 'year', 'month', 'day_of_the_week']].head()

,track_name,year,month,day_of_the_week
0,Without Me,2002,May,Sunday
1,The Real Slim Shady,2000,May,Tuesday
2,Till I Collapse,2002,May,Sunday
3,Mockingbird,2004,November,Friday
5,Lose Yourself,2005,December,Tuesday


In [101]:
df['featured_artists'] = df['featured_artists'].astype(str)
df['featured_artists'] = df['featured_artists'].replace({"'": '', '"': '', r'\[':'', r'\]':''}, regex=True)
df[['track_name', 'featured_artists']].head()

,track_name,featured_artists
0,Without Me,
1,The Real Slim Shady,
2,Till I Collapse,Nate Dogg
3,Mockingbird,
5,Lose Yourself,


In [102]:
key_dict = {
    0: "C",
    1: "C#/Db",
    2: "D",
    3: "D#/Eb",
    4: "E",
    5: "F",
    6: "F#/Gb",
    7: "G",
    8: "G#/Ab",
    9: "A",
    10: "A#/Bb",
    11: "B",
    -1: "NaN"
}

df['key'] = df['key'].map(key_dict).fillna("NaN")
df[['track_name', 'key']].head()

,track_name,key
0,Without Me,G
1,The Real Slim Shady,F
2,Till I Collapse,C#/Db
3,Mockingbird,C
5,Lose Yourself,D


In [104]:
df.to_csv('Eminem Dataset.csv', index=False)